In [1]:
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T
import pickle
import pandas as pd
import geopandas as gpd

import gnn_io

# Abstract

This is the new version of the the notebook "wandb_two_channel_gnn". 

What we do in here:

1. Load wandb and set reasonable parameters.
2. Introduce key parameters: a. Car Volume in area where there was change, b. Car volume overall, c. Car volume per Edge.
3. Make predictions for these parameters. 

In [2]:
# Load the dictionary
with open('../results/result_dic.pkl', 'rb') as f:
    results_dict = pickle.load(f)

all_car_volumes = []
all_policies = []
for key, df in results_dict.items():    
    if isinstance(df, pd.DataFrame):
        gdf = gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
        gdf.to_crs("EPSG:4326", inplace=True)
        edge_index, car_volume_tensor, policy_tensor, nodes = gnn_io.create_edge_index_and_tensors(gdf)
        all_car_volumes.append(car_volume_tensor)
        all_policies.append(policy_tensor)
    else:
        print(f"The value for key '{key}' is not a GeoDataFrame.")
        
final_car_volume_tensor = torch.cat(all_car_volumes, dim=0)
final_policy_tensor = torch.cat(all_policies, dim=0)

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


True

In [3]:
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.transforms import LineGraph

data = Data(edge_index = edge_index)
linegraph_transformation = LineGraph()
linegraph_data = linegraph_transformation(data)

print("Original graph edge index:\n", data.edge_index.shape)
print("Line graph edge index:\n", linegraph_data.edge_index.shape)
print("Line graph nodes: " + str(linegraph_data.num_nodes))

/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Original graph edge index:
 torch.Size([2, 31216])
Line graph edge index:
 torch.Size([2, 59183])
Line graph nodes: 31085


In [4]:
# X: Policy vector
# y: Flow targets

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            X = torch.from_numpy(X)
            y = torch.from_numpy(y)
        self.X = X
        self.y = y
            
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]
    
dataset = MyDataset(final_policy_tensor, final_car_volume_tensor)

In [5]:
dataset

In [6]:
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 10, shuffle = True, num_workers = 1) 

In [7]:
trainloader

In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [9]:
class GnnModel(nn.Module):
    def __init__(self):
        super(GnnModel, self).__init__()
        self.layers = nn.Sequential(
        nn.Linear(3, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1)
        )
        
    def forward(self, x):
        return self.layers(x)
    
model = GnnModel()

In [10]:
# Define the loss function and optimizer
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [11]:
import tqdm 
import torch.nn.functional as F

def train(model, train_loader, optimizer, epoch):
    model.train()
    total_loss = 0
    for data, target in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        
        loss.backward()
        total_loss += loss.item()
        
        optimizer.step()
    
    print("Train Epoch: {}\t Loss: {:.6f}".format(epoch, total_loss / len(train_loader)))

In [12]:
# import tqdm
for epoch in range(5):
    train(model= model, train_loader=trainloader, optimizer=optimizer, epoch = epoch)

  0%|          | 0/327768 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'MyDataset' on <module '__main__' (built-in)>
  0%|          | 0/327768 [00:01<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 11999) exited unexpectedly